In [9]:
from datasets import load_dataset

dataset = load_dataset('Kili/plastic_in_river', num_proc=6)

print(dataset)


Found cached dataset plastic_in_river (/Users/kevin/.cache/huggingface/datasets/Kili___plastic_in_river/default/1.3.0/9f50c1fffe85018c95137b17f0e5a271be58507e17e7a3f3a4075f763c20b8de)
100%|██████████| 3/3 [00:00<00:00, 19.10it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'litter'],
        num_rows: 3407
    })
    test: Dataset({
        features: ['image', 'litter'],
        num_rows: 427
    })
    validation: Dataset({
        features: ['image', 'litter'],
        num_rows: 425
    })
})


In [ ]:
import os
from tqdm.notebook import tqdm  # Import tqdm from tqdm.notebook module

# only creating datasets for train and validation not test
os.makedirs('datasets/images/train', exist_ok=True)
os.makedirs('datasets/images/test', exist_ok=True)
os.makedirs('datasets/images/validation', exist_ok=True)


os.makedirs('datasets/labels/train', exist_ok=True)
os.makedirs('datasets/labels/test', exist_ok=True)
os.makedirs('datasets/labels/validation', exist_ok=True)

def create_dataset(data, split):
    data = data[split]
  
    print(f'Running for {split} split...')
  
    for idx, sample in enumerate(data):
        image = sample['image']
        labels = sample['litter']['label']
        bboxes = sample['litter']['bbox']
        targets = []
        if(len(labels) > 0):
            class_id = 0
        else:
            class_id = 2
    
    # creating the label txt files
    for label, bbox in zip(labels, bboxes):
        targets.append(f'{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {class_id}')
      
    with open(f'datasets/labels/{split}/{idx}.txt', 'w') as f:
        for target in targets:
            f.write(target + '\n')
        
    # saving image to png
    image.save(f'datasets/images/{split}/{idx}.png')

create_dataset(dataset, 'train')
create_dataset(dataset, 'test')
create_dataset(dataset, 'validation')